In [ ]:
print("ok")

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.chroma import Chroma
from langchain_core.vectorstores import VectorStoreRetriever
# from langchain.vectorstores import Chroma
# import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
# from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from huggingface_hub.file_download import hf_hub_download
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
# from ctransformers.transformers import CTransformersModel
from langchain.chains import RetrievalQA

In [ ]:
# loader=DirectoryLoader(r"../data/", glob="*.pdf", loader_cls=PyPDFLoader)

In [ ]:
# document = loader.load()

In [ ]:
# text_splitter=RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=20)

In [ ]:
# text=text_splitter.split_documents(document)

In [ ]:
embedding=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
# vectordb=Chroma.from_documents(documents=text,embedding=embedding,persist_directory="db")
# vectordb.persist()

In [ ]:
vectordb=Chroma(persist_directory="db",embedding_function=embedding)

In [ ]:
retriever=vectordb.as_retriever(search_type="mmr",search_kwargs={"k":7})
# docs=retriever.get_relevant_documents("How to diagnose a patient?")
# docs
# retriever = VectorStoreRetriever(vectorstore=vectordb,search_type="mmr",search_kwargs={"k":4})
# docs=retriever.get_relevant_documents("How to diagnose a patient?")
# docs

In [ ]:
prompt="""
Use the following pieces of information to answer the user's question. If you don't know the answer, just say that you don't know, don't make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
prompt_template=PromptTemplate(
    input_variables=["context","question"],
    template=prompt
)
chain_type_kwargs={"prompt":prompt_template}

In [ ]:
# model_path=hf_hub_download(repo_id="TheBloke/Llama-2-7B-Chat-GGML", filename="llama-2-7b-chat.ggmlv3.q4_0.bin")
# model_path
# llm=CTransformers(
#     model=model_path,
#     model_type="llama",
#     config={
#         'max_new_tokens':512,
#         'temperature':0.8
#     }
# )

In [ ]:
llm=CTransformers(
    model="../model/llama-2-7b-chat.ggmlv3.q4_0.bin",
    model_type="llama",
    config={
        'max_new_tokens':512,
        'temperature':0.8
    }
)

In [ ]:
qa = RetrievalQA.from_chain_type(    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)
# qa = RetrievalQA.from_llm(llm,retriever)
# retrievalQA = RetrievalQA.from_llm(llm=llm, retriever=retriever)

In [ ]:
# while True:
print("executing")
user_input=input(f"Input prompt:")
result=qa({"query": user_input})
print("Response : ",result["result"])